### A. **Preço da ureia + commodities correlatas (base central)**

* **World Bank - Commodity Price Data (“Pink Sheet”)**: traz série **mensal** de preços de commodities, incluindo **fertilizantes (ureia)** e também **gás natural, petróleo, grãos, outros fertilizantes**, etc. Ótima para montar features consistentes e alinhadas em frequência.
* (Alternativa) **IMF Primary Commodity Prices**: também tem fertilizantes e séries de referência.

**Por que é “a base”**: com ela você já cobre direto vários itens da lista do cliente: gás, petróleo, grãos, nitrogenados substitutos e até proxies de energia.

---

### B. **Câmbio (para preço local e efeito de importação)**

* **BCB PTAX (API OData)**: cotações diárias (compra/venda) e você agrega para mensal (média/último dia útil).

---

### C. **Fretes / logística (proxy robusta e mensal)**

* **NY Fed - Global Supply Chain Pressure Index (GSCPI)**: índice mensal que incorpora custos de transporte (inclui medidas baseadas em frete marítimo como BDI/Harpex) e variáveis de oferta. Serve como proxy muito boa para “frete marítimo / gargalos”.

---

### D. **Geopolítica (guerras, sanções, tensões, tarifas)**

* **Geopolitical Risk Index (GPR)** (Caldara & Iacoviello): série **mensal** amplamente usada como proxy quantitativa de risco geopolítico (guerras/tensão/sanções).
* (Opcional) **Economic Policy Uncertainty (EPU)** via FRED para “política / tarifas / incerteza macro” (também mensal).

---

### E. **Clima (chuvas / ENSO como proxy global)**

* **NOAA ONI (Oceanic Niño Index)**: série mensal em CSV (ENSO), boa proxy de variações climáticas com impacto em agricultura/demanda logística.
---

### F. **Trade flows (China exportação, Índia import/tenders) - opcional**

* **UN Comtrade / WITS**: dá para extrair exportações/importações de ureia (ex.: China) e usar como feature (volume/valor), mas automatização pode exigir mais “engenharia”.

---


In [ ]:
import os
import re
import json
from io import BytesIO
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("All necessary libraries imported successfully.")

## Utilitários + Configs
---

In [ ]:
WORLD_BANK_PINK_SHEET_MONTHLY_XLSX = (
    "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
)

NYFED_GSCPI_XLSX = (
    "https://www.newyorkfed.org/medialibrary/research/interactives/gscpi/downloads/gscpi_data.xlsx"
)

NOAA_ONI_CSV = "https://psl.noaa.gov/data/correlation/oni.csv"

GPR_XLS = "https://www.matteoiacoviello.com/gpr_files/data_gpr_export.xls"

# BCB PTAX (OData) - exemplo comum com parâmetros:
# https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2020'&@dataFinalCotacao='12-31-2050'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao
BCB_PTAX_BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"


@dataclass
class SeriesSpec:
    out_name: str
    patterns: List[str]  # regex list to find a column in Pink Sheet


# Tentei deixar genérico o suficiente para sobreviver a pequenas mudanças de header.
PINK_SHEET_SERIES: List[SeriesSpec] = [
    SeriesSpec("urea_usd", [r"\burea\b"]),
    SeriesSpec("natural_gas_usd", [r"natural\s*gas", r"\bng\b"]),
    SeriesSpec("crude_oil_usd", [r"crude.*oil", r"\bbrent\b", r"\bwt[i|l]\b"]),
    SeriesSpec("maize_usd", [r"\bmaize\b", r"\bcorn\b"]),
    SeriesSpec("wheat_usd", [r"\bwheat\b"]),
    SeriesSpec("soybeans_usd", [r"\bsoy\b", r"\bsoybeans?\b"]),
    SeriesSpec("ammonia_usd", [r"\bammonia\b"]),
    SeriesSpec("dap_usd", [r"\bdap\b", r"diammonium\s*phosphate"]),
    SeriesSpec("potassium_usd", [r"\bpotassium\b"]),
]


def _safe_mkdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def _http_get(url: str, timeout: int = 60) -> bytes:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.content


def _to_month_start(dt: pd.Series) -> pd.Series:
    d = pd.to_datetime(dt, errors="coerce")
    return d.dt.to_period("M").dt.to_timestamp(how="start")


def _month_index(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[date_col] = _to_month_start(df[date_col])
    df = df.dropna(subset=[date_col])
    return df.set_index(date_col).sort_index()


def _pick_best_column(columns: List[str], patterns: List[str]) -> Optional[str]:
    cols_norm = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in columns}
    for pat in patterns:
        rx = re.compile(pat, flags=re.IGNORECASE)
        matches = [c for c, cn in cols_norm.items() if rx.search(cn)]
        if len(matches) == 1:
            return matches[0]
        if len(matches) > 1:
            # Heurística: se tiver "urea" e "gulf"/"bulk"/"granular", etc, escolha o mais descritivo
            # Caso não, escolha o primeiro em ordem alfabética (estável).
            matches_sorted = sorted(matches, key=lambda x: (len(str(x)), str(x)))
            return matches_sorted[0]
    return None


## Loaders
---
Com cache local em `/data/raw/` para evitar baixar os arquivos sempre.

In [ ]:
def load_pink_sheet_monthly(selected: List[SeriesSpec]) -> pd.DataFrame:
    # Cache local
    _safe_mkdir(os.path.join("data", "raw"))
    csv_path = os.path.join("data", "raw", "pink_sheet_monthly.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, parse_dates=["date"])
        return _month_index(df, "date")

    # Buscar da web e salvar
    content = _http_get(WORLD_BANK_PINK_SHEET_MONTHLY_XLSX)
    xls = pd.ExcelFile(BytesIO(content))

    # Normalmente o primeiro sheet já é o "Monthly Prices", mas deixamos robusto:
    sheet_name = xls.sheet_names[1]
    df_raw = pd.read_excel(xls, sheet_name=sheet_name, engine="openpyxl", skiprows=4, header=[0, 1])

    # Flatten MultiIndex columns: juntar nome e unidade com espaço
    df_raw.columns = [' '.join(col).strip() for col in df_raw.columns.values]

    # Descobrir coluna de data (alguns arquivos usam "date" / "Month" / "Time")
    possible_date_cols = [c for c in df_raw.columns if str(c).strip().lower() in ("date", "time", "month")]
    if not possible_date_cols:
        # fallback: primeira coluna
        date_col = df_raw.columns[0]
    else:
        date_col = possible_date_cols[0]

    df = df_raw.rename(columns={date_col: "date"}).copy()
    df["date"] = pd.to_datetime(df["date"].astype(str).str.replace('M', ''), format='%Y%m', errors='coerce')

    # Selecionar séries por regex
    picked = {}
    missing = []
    for spec in selected:
        col = _pick_best_column(list(df.columns), spec.patterns)
        if col is None:
            missing.append(spec.out_name)
            continue
        picked[col] = spec.out_name

    if missing:
        print("[AVISO] Algumas séries não foram encontradas no Pink Sheet:", missing)
        print("[DEBUG] Colunas disponíveis (amostra):", list(df.columns)[:30])

    out = df[["date"] + list(picked.keys())].rename(columns=picked)

    # Converter tudo para numérico (algumas colunas podem vir como object)
    for c in out.columns:
        if c != "date":
            out[c] = pd.to_numeric(out[c], errors="coerce")

    out_idx = _month_index(out, "date")
    out_idx.reset_index().to_csv(csv_path, index=False)
    return out_idx


def load_bcb_ptax_usdbrl(date_start: str, date_end: str) -> pd.DataFrame:
    # A API usa formato dd-mm-aaaa nas strings do parâmetro
    # Vamos aceitar start/end como "YYYY-MM" ou "YYYY-MM-DD" e converter.
    start_dt = pd.to_datetime(date_start) if len(date_start) > 7 else pd.to_datetime(date_start + "-01")
    end_dt = pd.to_datetime(date_end) if len(date_end) > 7 else (pd.to_datetime(date_end + "-01") + pd.offsets.MonthEnd(0))

    start_str = start_dt.strftime("%m-%d-%Y")  # muitos exemplos aceitam MM-DD-YYYY
    end_str = end_dt.strftime("%m-%d-%Y")

    url = (
        f"{BCB_PTAX_BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
        f"?@dataInicial='{start_str}'&@dataFinalCotacao='{end_str}'&$format=json"
        f"&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
    )

    # Cache por janela de datas
    _safe_mkdir(os.path.join("data", "raw"))
    safe_start = date_start.replace("/", "-").replace(" ", "_")
    safe_end = date_end.replace("/", "-").replace(" ", "_")
    csv_path = os.path.join("data", "raw", f"bcb_ptax_usdbrl_{safe_start}_{safe_end}.csv")
    if os.path.exists(csv_path):
        df_m = pd.read_csv(csv_path, parse_dates=["date"])
        return _month_index(df_m, "date")

    data = _http_get(url)
    j = json.loads(data.decode("utf-8"))
    values = j.get("value", [])
    df = pd.DataFrame(values)
    if df.empty:
        raise RuntimeError("BCB PTAX: retorno vazio. Verifique janela de datas/URL.")

    df["date"] = pd.to_datetime(df["dataHoraCotacao"], errors="coerce")
    df["usdbrl"] = pd.to_numeric(df["cotacaoVenda"], errors="coerce")

    # Agregar para mensal (média)
    df_m = (
        df.dropna(subset=["date", "usdbrl"])
          .assign(date=_to_month_start(df["date"]))
          .groupby("date", as_index=False)["usdbrl"]
          .mean()
    )
    df_m.reset_index(drop=True)
    df_m.reset_index().to_csv(csv_path, index=False)
    return _month_index(df_m, "date")


def load_nyfed_gscpi() -> pd.DataFrame:
    _safe_mkdir(os.path.join("data", "raw"))
    csv_path = os.path.join("data", "raw", "nyfed_gscpi.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, parse_dates=["date"])
        return _month_index(df, "date")

    content = _http_get(NYFED_GSCPI_XLSX)
    xls = pd.ExcelFile(BytesIO(content))
    # Em geral há duas sheet com a série e coluna "GSCPI"
    sheet = xls.sheet_names[1]
    df = pd.read_excel(xls, sheet_name=sheet)

    # Tentar inferir colunas:
    date_col = _pick_best_column(list(df.columns), [r"date", r"month", r"time"]) or df.columns[0]
    val_col = _pick_best_column(list(df.columns), [r"gscpi"]) or df.columns[1]

    out = df.rename(columns={date_col: "date", val_col: "gscpi"})[["date", "gscpi"]].copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out["gscpi"] = pd.to_numeric(out["gscpi"], errors="coerce")
    out_idx = _month_index(out, "date")
    out_idx.reset_index().to_csv(csv_path, index=False)
    return out_idx


def load_noaa_oni() -> pd.DataFrame:
    _safe_mkdir(os.path.join("data", "raw"))
    csv_path = os.path.join("data", "raw", "noaa_oni.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, parse_dates=["date"])
        return _month_index(df, "date")

    content = _http_get(NOAA_ONI_CSV)
    df = pd.read_csv(BytesIO(content))
    # Esperado: date, ONI
    date_col = _pick_best_column(list(df.columns), [r"date"]) or df.columns[0]
    val_col = _pick_best_column(list(df.columns), [r"oni"]) or df.columns[1]
    out = df.rename(columns={date_col: "date", val_col: "oni"})[["date", "oni"]].copy()
    out["date"] = pd.to_datetime(out["date"], errors="coerce")
    out["oni"] = pd.to_numeric(out["oni"], errors="coerce")
    out_idx = _month_index(out, "date")
    out_idx.reset_index().to_csv(csv_path, index=False)
    return out_idx


def load_gpr() -> pd.DataFrame:
    _safe_mkdir(os.path.join("data", "raw"))
    csv_path = os.path.join("data", "raw", "gpr.csv")
    gpr_dict_path = os.path.join("data", "raw", "gpr_dict.csv")
    if os.path.exists(csv_path) and os.path.exists(gpr_dict_path):
        df = pd.read_csv(csv_path, parse_dates=["date"]).set_index("date")
        dictionary = pd.read_csv(gpr_dict_path)
        return (_month_index(df.reset_index(), "date"), dictionary)

    content = _http_get(GPR_XLS)

    try:
        df = pd.read_excel(BytesIO(content), engine="xlrd")
    except ImportError as e:
        raise ImportError(
            "Para ler arquivos .xls, instale xlrd: pip install xlrd"
        ) from e
    except ValueError:
        # fallback: tenta sem engine explicita
        df = pd.read_excel(BytesIO(content))

    # Normaliza nomes
    cols_norm = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in df.columns}

    # --- 1) Construir coluna date ---
    # Caso A: tem year e month separados
    year_col = next((c for c, cn in cols_norm.items() if cn in ("year", "yr", "yyyy")), None)
    month_col = next((c for c, cn in cols_norm.items() if cn in ("month", "mo", "mm")), None)

    if year_col is not None and month_col is not None:
        out = df[[year_col, month_col]].copy()
        out["date"] = pd.to_datetime(
            dict(year=pd.to_numeric(out[year_col], errors="coerce"),
                 month=pd.to_numeric(out[month_col], errors="coerce"),
                 day=1),
            errors="coerce",
        )
        df2 = df.copy()
        df2["date"] = out["date"]
    else:
        # Caso B: tem date / time / month como coluna única
        date_col = _pick_best_column(list(df.columns), [r"date", r"time", r"month"]) or df.columns[0]
        df2 = df.rename(columns={date_col: "date"}).copy()
        # Se vier como YYYYMM numérico (ex.: 202112), converte
        if pd.api.types.is_numeric_dtype(df2["date"]):
            yyyymm = pd.to_numeric(df2["date"], errors="coerce")
            year = (yyyymm // 100).astype("Int64")
            month = (yyyymm % 100).astype("Int64")
            df2["date"] = pd.to_datetime(dict(year=year, month=month, day=1), errors="coerce")
        else:
            df2["date"] = pd.to_datetime(df2["date"], errors="coerce")
    
    dictionary = pd.DataFrame()
    if os.path.exists(gpr_dict_path):
        dictionary = pd.read_csv(gpr_dict_path, parse_dates=["date"]).set_index("date")

    if dictionary.empty:
        dictionary = df2[["var_name", "var_label"]].dropna().copy()
        dictionary.loc[0, "var_name"] = "date"
        dictionary["var_name"] = dictionary["var_name"].apply(str.lower)
        dictionary.to_csv(gpr_dict_path, index=False)

    # --- 2) Selecionar colunas do GPR ---
    # Ele pode ter GPR agregado e também decomposições (ameaça/ato etc.). Vamos puxar tudo que começar com gpr
    gpr_cols = [c for c, cn in cols_norm.items() if cn == "gpr" or cn.startswith("gpr")]

    if not gpr_cols:
        # fallback: tenta achar pelo regex no nome original
        gpr_cols = [c for c in df2.columns if re.search(r"(^|\W)gpr(\W|$)", str(c), flags=re.IGNORECASE)]

    if not gpr_cols:
        raise RuntimeError("Não encontrei nenhuma coluna 'GPR' no arquivo data_gpr_export.xls.")

    keep = ["date"] + gpr_cols
    out = df2[keep].copy()

    # Numeric
    for c in gpr_cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")

    # Padroniza nomes (opcional)
    rename_map = {}
    for c in gpr_cols:
        cn = cols_norm.get(c, str(c).lower())
        # deixa nomes amigáveis
        rename_map[c] = re.sub(r"[^a-z0-9_]+", "_", cn).strip("_")
    out = out.rename(columns=rename_map)

    out_idx = _month_index(out, "date")
    out_idx.reset_index().to_csv(csv_path, index=False)
    return (out_idx, dictionary)

def load_events():
    _safe_mkdir(os.path.join("data", "raw"))
    csv_path = os.path.join("data", "raw", "main_events.csv")
    df = pd.read_csv(csv_path)
    df["date"] = df['period'].str[:7]
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m", errors="coerce")
    df = df[["period", "date", "event"]]
    df.to_csv(csv_path, index=False)

    return _month_index(df, "date")

## Definindo os parâmetros:
---

In [ ]:
OUTDIR = os.path.abspath("output")
DATA_DIR = os.path.abspath("data")
START_DATE = "1990-01"
END_DATE = "2025-12"
TARGET = "Urea  ($/mt)"

## Executando os loaders:
---

In [ ]:
_safe_mkdir(OUTDIR)
figdir = os.path.join(OUTDIR, "figures")
_safe_mkdir(figdir)

# 1) Loaders
print("Baixando Pink Sheet (World Bank)...")
df_prices = load_pink_sheet_monthly(PINK_SHEET_SERIES)

print("Baixando câmbio PTAX (BCB)...")
df_fx = load_bcb_ptax_usdbrl(START_DATE, END_DATE)

print("Baixando GSCPI (NY Fed)...")
df_gscpi = load_nyfed_gscpi()

print("Baixando GPR (Geopolitical Risk)...")
df_gpr, gpr_dict = load_gpr()

print("Baixando ONI (NOAA)...")
df_oni = load_noaa_oni()

print("Carregando eventos principais...")
df_events = load_events()

## Unificando todas as tabelas em uma só:
---

In [ ]:
df_gpr_small = (
    df_gpr[
        [
            "gpr",
            "gprt",
            "gpra",
            "gprh",
            "gprht",
            "gprha",
            "gprc_bra",
            "gprhc_bra",
            "gprc_chn",
            "gprhc_chn",
            "gprc_ind",
            "gprhc_ind"
        ]
    ].copy()
)
df = (
    df_prices
    .merge(df_fx, how="outer", left_index=True, right_index=True)
    .merge(df_gscpi, how="outer", left_index=True, right_index=True)
    .merge(df_gpr_small, how="outer", left_index=True, right_index=True)
    .merge(df_oni, how="outer", left_index=True, right_index=True)
    .merge(df_events, how="outer", left_index=True, right_index=True)
)
df = df.loc[(df.index >= pd.to_datetime(START_DATE + "-01" if len(START_DATE) == 7 else START_DATE)) &
            (df.index <= pd.to_datetime(END_DATE + "-01" if len(END_DATE) == 7 else END_DATE) + pd.offsets.MonthEnd(0))]

df.to_csv(os.path.join(DATA_DIR, "merged_data.csv"), index=True)